In [1]:
import pyodbc #подключение к SQL Server
import openpyxl #Работа с файлами excel
import pathlib #Работа с файлами и путями к ним
 #import user_agents #Работа с данными пользователя, например, выделение браузера
import pandas as pd #Работа с табличными данными
import matplotlib.pyplot as plt #Работа с графиками
import datetime #Работа с датами
import seaborn as sbnm #Работа с графиками
import numpy as np 
from scipy import stats as st
import openpyxl as ox
from openpyxl import Workbook
from openpyxl.worksheet import worksheet
import os
from pandas.io.excel import ExcelWriter
import json
import warnings  #Выключение предупреждений

In [2]:
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore')

Driver='ODBC Driver 17 for SQL Server'
Server=r'srv-dwh-lsn, 15025'
Database=['dwh','Reps','presentation']
# 'dwh' = Database[0]
# 'Reps' = Database[1]
# 'presentation' = Database[2]

def get_adw_connection():
    # Установка соединения 
    connection = pyodbc.connect('Driver=' + Driver + ';\
                                 Server=' + Server + ';\
                                 Database=' + Database[0] + ';\
                                 Trusted_Connection=yes')
    return connection



**ЗАГРУЖАЕМ ИЗ EXCEL**

In [3]:
quots = pd.read_excel('C:\\Users\\VYuYaroshenko\\Desktop\\Пайтон ТАБАК\\quots.xlsx')
quots.head()

,level3,plng,size,qt,rtn
0,ТАБАК,ХОЗ.ТОВАРЫ,0,1,DT4
1,ТАБАК,ХОЗ.ТОВАРЫ,1,1,DT4
2,ТАБАК,ХОЗ.ТОВАРЫ,2,1,DT4
3,ТАБАК,ХОЗ.ТОВАРЫ,3,1,DT4
4,ТАБАК,ХОЗ.ТОВАРЫ,4,1,DT4


**ЗАГРУЖАЕМ ИЗ БД**

In [4]:
%%time
#SET NOCOUNT ON; 

query = """

SELECT 
PlngDescription 'Plng',
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end 'FO', 
s.level4_descr 'Область', D.g_ItemCode 'Item', PlngSize, left(PlngSize,1) 'Claster', 
case when PlngSize='G20' then '20'
	 when PlngSize='D77' then '77' else right(PlngSize,1) end 'Size',
count(DISTINCT s.SiteExtID) 'Count_TT'

FROM 
[DWH].[GLD].[TypeSettingMatrixPeriod] mp
JOIN [DWH].[GLD].[Attributes] a on mp.AttributeID = a.AttributeId
JOIN [DWH].[dbo].[DProducts] d on mp.ItemId = d.ItemId
JOIN [DWH].[dbo].[DNodes] s on mp.SiteId = s.SiteID
    
WHERE 1=1 and format(getdate(), 'yyyyMMdd') between mp.DateFrom and mp.DateTo and 
InOut in ('Постоянный', 'Временный') and AssortTypeName in ('РЕГУЛЯРНЫЙ', 'СЕЗОННЫЙ','РАСПРОДАЖА')
and concat(InOut,' ',AssortTypeName) != 'Временный РЕГУЛЯРНЫЙ'
and D.Level3_desc = 'ТАБАК'
and left(PlngSize,1) = 'D'

GROUP BY
PlngDescription ,
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end , 
s.level4_descr , PlngSize, D.g_ItemCode , left(PlngSize,1) , 
case when PlngSize='G20' then '20' when PlngSize='D77' then '77' else right(PlngSize,1) end 

order by 3

"""

gp=get_adw_connection()
now_tt = pd.read_sql(query, con=gp)
gp.close()



query = """

SELECT 
PlngDescription 'Plng',
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end 'FO', 
s.level4_descr 'Область', PlngSize, left(PlngSize,1) 'Claster', 
case when PlngSize='G20' then '20' when PlngSize='D77' then '77' else right(PlngSize,1) end 'Size',
count(DISTINCT D.g_ItemCode) 'Count_Item'

FROM 
[DWH].[GLD].[TypeSettingMatrixPeriod] mp
JOIN [DWH].[GLD].[Attributes] a on mp.AttributeID = a.AttributeId
JOIN [DWH].[dbo].[DProducts] d on mp.ItemId = d.ItemId
JOIN [DWH].[dbo].[DNodes] s on mp.SiteId = s.SiteID
    
WHERE 1=1 and format(getdate(), 'yyyyMMdd') between mp.DateFrom and mp.DateTo and 
InOut in ('Постоянный', 'Временный') and AssortTypeName in ('РЕГУЛЯРНЫЙ', 'СЕЗОННЫЙ','РАСПРОДАЖА')
and concat(InOut,' ',AssortTypeName) != 'Временный РЕГУЛЯРНЫЙ'
and D.Level3_desc = 'ТАБАК'

GROUP BY
PlngDescription ,
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end , 
s.level4_descr , PlngSize, left(PlngSize,1) , case when PlngSize='G20' then '20' when PlngSize='D77' then '77' else right(PlngSize,1) end 

order by 3

"""

gp=get_adw_connection()
now_sku = pd.read_sql(query, con=gp)
gp.close()

CPU times: total: 344 ms
Wall time: 14.2 s


In [5]:
print('----------------квоты----------------------------------------------------------------------------------------')
print(quots.head(3))
#print(' ')
#print('----------------матрица с кол-вом магазинов присутствия------------------------------------------------------')
#print(now_tt.head(3))
print(' ')
print('----------------матрица с кол-вом sku присутствия------------------------------------------------------------')
print(now_sku.head(3))

----------------квоты----------------------------------------------------------------------------------------
  level3        plng  size  qt  rtn
0  ТАБАК  ХОЗ.ТОВАРЫ     0   1  DT4
1  ТАБАК  ХОЗ.ТОВАРЫ     1   1  DT4
2  ТАБАК  ХОЗ.ТОВАРЫ     2   1  DT4
 
----------------матрица с кол-вом sku присутствия------------------------------------------------------------
                Plng   FO           Область PlngSize Claster Size  Count_Item
0           ПРИКАССА  ЦФО  БРЯНСКАЯ ОБЛАСТЬ       L5       L    5           3
1           ПРИКАССА  ЦФО  БРЯНСКАЯ ОБЛАСТЬ       M5       M    5           3
2  ТАБ. ПК ДИСПЕНСЕР  ЦФО  БРЯНСКАЯ ОБЛАСТЬ       M1       M    1          36


In [6]:
now_sku #271 rows

,Plng,FO,Область,PlngSize,Claster,Size,Count_Item
0,ПРИКАССА,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,L5,L,5,3
1,ПРИКАССА,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M5,M,5,3
2,ТАБ. ПК ДИСПЕНСЕР,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M1,M,1,36
3,ТАБАЧНЫЙ ДИСПЕНСЕР,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M1,M,1,73
4,ТАБАЧНЫЙ ШКАФ,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M1,M,1,45
...,...,...,...,...,...,...,...
411,ХОЗ.ТОВАРЫ,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,L2,L,2,1
412,ПРИКАССА,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,L2,L,2,3
413,ПРИКАССА,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,M2,M,2,3
414,ХОЗ.ТОВАРЫ,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,M2,M,2,1


In [7]:
quots = quots.rename(columns = {'plng':'Plng', 'size':'Size', 'qt':'Quot'})
quots_1 = quots[['Plng', 'Size', 'Quot']]
now_sku['Size'] = now_sku['Size'].astype(int)
quots_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Plng    24 non-null     object
 1   Size    24 non-null     int64 
 2   Quot    24 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 704.0+ bytes


In [8]:
now_quots = pd.merge(now_sku, quots_1, on=['Plng', 'Size'], how='left') #271 rows
now_quots

,Plng,FO,Область,PlngSize,Claster,Size,Count_Item,Quot
0,ПРИКАССА,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,L5,L,5,3,29.0
1,ПРИКАССА,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M5,M,5,3,29.0
2,ТАБ. ПК ДИСПЕНСЕР,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M1,M,1,36,36.0
3,ТАБАЧНЫЙ ДИСПЕНСЕР,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M1,M,1,73,72.0
4,ТАБАЧНЫЙ ШКАФ,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M1,M,1,45,120.0
...,...,...,...,...,...,...,...,...
411,ХОЗ.ТОВАРЫ,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,L2,L,2,1,1.0
412,ПРИКАССА,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,L2,L,2,3,29.0
413,ПРИКАССА,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,M2,M,2,3,29.0
414,ХОЗ.ТОВАРЫ,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,M2,M,2,1,1.0


In [9]:
now_quots['Quot'] = now_quots['Quot'].fillna(0)
now_quots ['Отклонение'] = now_quots['Quot'] - now_quots ['Count_Item']
data  = now_quots[['Plng', 'FO', 'Область', 'Claster', 'Size', 'Отклонение', 'Count_Item', 'Quot']]
data

,Plng,FO,Область,Claster,Size,Отклонение,Count_Item,Quot
0,ПРИКАССА,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,L,5,26.0,3,29.0
1,ПРИКАССА,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M,5,26.0,3,29.0
2,ТАБ. ПК ДИСПЕНСЕР,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M,1,0.0,36,36.0
3,ТАБАЧНЫЙ ДИСПЕНСЕР,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M,1,-1.0,73,72.0
4,ТАБАЧНЫЙ ШКАФ,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M,1,75.0,45,120.0
...,...,...,...,...,...,...,...,...
411,ХОЗ.ТОВАРЫ,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,L,2,0.0,1,1.0
412,ПРИКАССА,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,L,2,26.0,3,29.0
413,ПРИКАССА,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,M,2,26.0,3,29.0
414,ХОЗ.ТОВАРЫ,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,M,2,0.0,1,1.0


**EXCEL**

In [10]:
file_path="C:\\Users\\VYuYaroshenko\\Desktop\\Пайтон ТАБАК\\данные.xlsx" # путь к файлу

#запись в Excel-файл датафрейма now_quots: 
data.to_excel(
    file_path, 
    sheet_name='выгрузка', # название листа Сегодня
    index=False, # не будут указаны индексы
    startrow=0, # место записи с 0й строки
    startcol=0 # место записи с 0й колонки
)